<a href="https://colab.research.google.com/github/FFabricio/Data-Set-for-Intravenous-Infusion-Profiles-/blob/master/InfusionProfiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import** from libraries

In [0]:
import math
import numpy as np
import pandas as pd
import os
import pickle 
import IPython
from google.colab import files


### **Loads machine learning models**

The models are available on the drive link.
Please upload to your google drive and change the file path.

https://drive.google.com/drive/folders/1sWwWhyRSimbMTZTDDGNiLvX-iSP1LfX6?usp=sharing




In [0]:
Bi_model = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Bi_model.sav', 'rb'))
Gra_model = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Gra_model.sav', 'rb'))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PolynomialFeatures from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Ridge from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. U

**Functions** that add real features to simulated data (noise)



In [0]:
def ruido_ml(sinal,tam):    
    vout_sgr=[]
    noise=[]
    sigpower=[]
    regsnr=54
    sigpower=sum([math.pow(abs(sinal[i]),2) for i in range(tam)])
    sigpower=sigpower/tam
    noisepower=sigpower/(math.pow(10,regsnr/10))
    noise=math.sqrt(noisepower)*(np.random.uniform(-1,1,size=tam))
    vout_sgr=[ noise[i]+sinal[i] for i in range(tam)]
    return vout_sgr    

**Function** manages simulation

In [0]:
def simula(escoa,janela,repeti):
  dado_bi= open('/content/DataSetIP.csv','w')
  dado_gra= open('/content/DataSetGra.csv','w')
  ensaio=1
  volume=125
  t_vol=1
  passo=escoa
  escoa=22
  while escoa<=999.9:
      if volume == 125:
          limite_tempo=.6
      if volume == 250:
          limite_tempo=.8
      if volume == 500:
          limite_tempo=.9
      if volume == 1000:
          limite_tempo=1
      if volume/escoa < limite_tempo:
          t_vol=t_vol+1
          if t_vol==2:
              volume=250
              escoa=72
          if passo<5: 
              passo=5                
          if t_vol==3:
              escoa=33
              volume=500
          if passo<15:
              passo=15
          if t_vol==4:
              escoa=64
              volume=1000 
          if passo<30:
              passo=30 
      for jk in range(repeti):
          volume_t=0
          while (volume_t<(volume*0.95) or volume_t>(volume)):   
              tol_v=np.float((np.random.randint(-25,25,size=(1)))/1000)
              tol_e=np.float((np.random.randint(-5,5,size=(1)))/1000)
              tempo=((1+tol_v)*volume)/((1+tol_e)*escoa)
              volume_t=round((tempo)*escoa*(1+tol_e),1)
          tempo=int(tempo*3600)
          escoa_t=round(escoa*(1+tol_e),1)
          data_t=[]
          data_e=[]
          data_v=[]
          c=[[data_t.append(i),data_v.append(volume_t), data_e.append(escoa_t)] for i in range(tempo)]              
          data=[]    
          data=pd.DataFrame(data_e,columns=['Flow'])    
          data['Volume']=pd.DataFrame(data_v)    
          data['Time']=pd.DataFrame(data_t) 
          dataGra0=Gra_model.predict(data[['Flow','Volume','Time']])
          dataGra=ruido_ml(dataGra0,len(dataGra0))
          dataBi0=Bi_model.predict(data[['Flow','Volume','Time']])   
          dataBi=ruido_ml(dataBi0,len(dataBi0))
          if ensaio==1:
            for i in range(0, janela):
              dado_gra.write('TimeWin'+str(i)+',')
              dado_bi.write('TimeWin'+str(i)+',')
            dado_gra.write('DrugDelivered,DrugReservoirs,FlowRate,Test\n')
            dado_bi.write('DrugDelivered,DrugReservoirs,FlowRate,Test\n')
          d1=[]
          d3=[]
          j=0
          tam=len(dataGra)
          for i in range(0,tam):
              data_reg=np.float(dataGra[i])
              d1.append("%0.6f"%data_reg)
              data_reg=np.float(dataBi[i])
              d3.append("%0.6f"%data_reg)
              if len(d1)==janela:
                  d1 = ",".join(d1)
                  dado_gra.write(d1+','+str(volume_t)+','+str(volume)+','+str(escoa_t)+','+str(ensaio)+'\n')
                  d3 = ",".join(d3)
                  dado_bi.write(d3+','+str(volume_t)+','+str(volume)+','+str(escoa_t)+','+str(ensaio)+'\n')                  
                  j=0
                  d1=[]
                  d3=[]
              j=j+1            
          ensaio=ensaio+1
          del dataGra0
          del dataGra
          del dataBi0
          del dataBi
      escoa=escoa+passo 
  files.download('DataSetGra.csv')
  files.download('DataSetIP.csv')
  dado_gra.close() 
  dado_bi.close()     
  return 

Data entry for the simulation
## **In case of errors, please consider the local use of the notebook!!!**

In [0]:
flag=1
while flag==1:
  flag=0
  print('Enter Increment Interval between [1.0 to 100.0] mL/h')
  try:
    escoa=float(input())
  except ValueError:
    flag=1
    print('Enter numbers only')
  if flag == 0:  
    if escoa>100:
      print('Maximum flow limited to 100 mL/h')
      flag=1
    if escoa<1:
      print('Minimum flow limited to 1 mL/h')
      flag=1
flag=1
while flag==1:
  flag=0
  print('\nEnter Time Window Size [1 to 360] Seconds')
  try:
    jan=int(input())
  except ValueError:
    flag=1
    print('Enter numbers only')
  if flag==0:  
    if jan>360:
      print('Maximum Time Window Size limited to 360 Seconds')
      flag=1
    if jan<1:
      print('Minimum Time Window Size limited to 1 Seconds')
      flag=1
flag=1
while flag==1:
  flag=0
  print('\nEnter the Number of Simulation Repeats [1 to 50]')
  try:
    repeti=int(input())
  except ValueError:
    flag=1
    print('Enter numbers only')
  if flag==0:  
    if repeti>50:
      print('Maximum the Number of Simulation Repeats is 50')
      flag=1
    if repeti<1:
      print('Minimum the Number of Simulation Repeats is 1')
      flag=1
print('Running the Simulation ...')
simula(int(escoa),int(jan),int(repeti))
print('End of Simulation.')

Enter Increment Interval between [1.0 to 100.0] mL/h
10

Enter Time Window Size [1 to 360] Seconds
60

Enter the Number of Simulation Repeats [1 to 50]
1
Running the Simulation ...
End of Simulation.
